# OFF-CHAIN SETUP, PROOF GENERATION AND VERIFICATION

## THE CIRCUIT CONFIGURATION

In [1]:
# Only for Colab
from google.colab import drive
drive.mount('/content/drive')

# Set the current working directory
import os
os.chdir('/content/drive/MyDrive/ml-verifier-oracle/prover')
print("Current path: " + os.getcwd())

Mounted at /content/drive
Current path: /content/drive/MyDrive/ml-verifier-oracle/prover


In [2]:
# Check if notebook is in colab
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# Rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import os
import json
import ezkl
import pandas as pd

import torch
from torchvision import transforms
from PIL import Image

import logging
import os
import sys
import sys
import time
from datetime import datetime

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(project_root)
from training.defineSNN import MODSiameseBambooNN
from training.defineAdjustedSNN import FurtherAdjustedSiameseBambooNN
from training.defineReducedSNN import ReducedParamsSiameseBambooNN

# Uncomment for more descriptive logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)


# Specify all the files we need
model_path = os.path.join('ezkl-outputs/snn.onnx')
compiled_model_path = os.path.join('ezkl-outputs/snn.ezkl')
pk_path = os.path.join('ezkl-outputs/pk.key')
vk_path = os.path.join('ezkl-outputs/vk.key')
settings_path = os.path.join('ezkl-outputs/settings.json')
srs_path = os.path.join('ezkl-outputs/kzg.srs')
data_path = os.path.join('ezkl-outputs/input.json')
cal_path = os.path.join('ezkl-outputs/cal_data.json')
##
sol_code_path = os.path.join('../contracts/Verifier.sol')
abi_path = os.path.join('ezkl-outputs/Verifier.abi')
proof_path = os.path.join('ezkl-outputs/proof.json')




In [3]:
# Check the ezkl version installed (>=7.1.12 has RUST log with all errors report)
!pip list | grep ezkl

ezkl                             7.1.12


In [4]:
# Assign benchmark test name each time a new test is executed to save its results into a file

timestamp = datetime.now().strftime("%m_%d_%H-%M")
test_name = f"test_015___{timestamp}"
test_time_and_size_path = os.path.join(f'../benchmark/{test_name}_time_and_size.txt')

print(test_name)

test_015___01_25_15-37


In [5]:
# Load the image pair as input ready for inference
class SiameseImageLoader:
    def __init__(self, transform=None):
        self.transform = transform

    def load_and_transform_pair(self, image1_path, image2_path):
        img1 = Image.open(image1_path).convert('L')
        img2 = Image.open(image2_path).convert('L')

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        img1 = img1.to(device)
        img2 = img2.to(device)

        return img1, img2

transform = transforms.Compose([
    transforms.Resize((10, 10)),
    transforms.ToTensor(),
])

# Example inputs (TrueNegative, TruePositive, FalseNegative)
# (FalseNegative were produced with a growing rate of bamboo higher than what the model was trained on)
siamese_loader = SiameseImageLoader(transform)
image1_path = "TestValid_t1.jpeg"
image2_path = "TestValid_t0.jpeg"
img1, img2 = siamese_loader.load_and_transform_pair(image1_path, image2_path) # Separate tensors for each image
img1 = img1.unsqueeze(0)
img2 = img2.unsqueeze(0)

print(img1.size())

torch.Size([1, 1, 10, 10])


In [6]:
# Load the trained nn model and make inference on test inputs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#######################
# Load the model and the parameters (state dictionary)
loaded_model = ReducedParamsSiameseBambooNN().to(device)
loaded_model.load_state_dict(torch.load('../training/trained_simplesnn_lr_0.01S.pth', map_location=torch.device('cpu')))
#######################
loaded_model.eval()

with torch.no_grad():
    finalout = loaded_model(img1, img2)
    predicted_label = (torch.sigmoid(finalout) > 0.5).item()

print(f'Value of model outcome before sigmoid: {finalout}')
print(f'Same class probability: {torch.sigmoid(finalout)}')
print(f'Predicted Label: {predicted_label}')

# Calculate the total number of parameters
total_params = sum(p.numel() for p in loaded_model.parameters())
print(f'Total Parameters: {total_params}')

Value of model outcome before sigmoid: tensor([[4.0211]], device='cuda:0')
Same class probability: tensor([[0.9824]], device='cuda:0')
Predicted Label: True
Total Parameters: 8545


In [7]:
# Export the loaded Siamese Network model to ONNX
torch.onnx.export(loaded_model,                      # loaded Siamese Network model
                  (img1, img2),                      # model input (or a tuple for multiple inputs)
                  model_path,                        # where to save the ONNX model
                  export_params=True,                # store the trained parameter weights inside the model file
                  opset_version=14,                  # the ONNX version to export the model to
                  do_constant_folding=True,          # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})



# Move tensors from GPU to CPU
img1_cpu = img1.cpu()
img2_cpu = img2.cpu()
print(img1_cpu.size())

# Serialize data into a JSON file
input_data = [
    img1_cpu.detach().numpy().reshape([-1]).tolist(),
    img2_cpu.detach().numpy().reshape([-1]).tolist(),
]

data = dict(input_data=input_data)
json.dump(data, open(data_path, 'w'))

torch.Size([1, 1, 10, 10])


In [8]:
# *** Providing the calibrate settings function with a larger and broader range of sample inputs.

path_to_dataset = '../training/dataset10k20x20'

csv_path = os.path.join(path_to_dataset, 'forest_dataset.csv')
df = pd.read_csv(csv_path)

# Construct the full paths to the images
img1_files = [os.path.join('..', 'training/dataset10k20x20', path) for path in df['imageT0'].iloc[:20]]
img2_files = [os.path.join('..', 'training/dataset10k20x20', path) for path in df['imageT1'].iloc[:20]]

img1_list = []
img2_list = []
for img1_path, img2_path in zip(img1_files, img2_files):
    img1, img2 = siamese_loader.load_and_transform_pair(img1_path, img2_path)
    img1 = img1.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img2 = img2.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img1_list.extend(img1) # do not use .append bc it should look like [[xxx...],[yyy...]]
    img2_list.extend(img2)

data = dict(input_data = [img1_list, img2_list])
# Serialize data into file:
json.dump( data, open(cal_path, 'w' ))

In [ ]:
# For ezkl to compute a snark, it needs some settings to determine how to create the circuit.
# This cell instantiates some parameters that determine the circuit shape, size etc
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs
py_run_args.variables = [("batch_size", 1)]

!RUST_LOG=trace

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True
print("gen_settings OK")

# Calibration, "resources" or "accuracy", scales and max_logrows can be set
res = ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources", scales = [1, 1])


INFO:ezkl.graph.model:set batch_size to 1
INFO:ezkl.graph.model:model has 1 instances
INFO:ezkl.graph.model:calculating num of constraints using dummy model layout...
INFO:ezkl.graph.model:model uses 8086 rows (coord=16173, constants=14254)
INFO:ezkl.graph.model:set batch_size to 1
INFO:ezkl.execute:num of calibration batches: 20
INFO:ezkl.execute:running onnx predictions...
INFO:ezkl.graph.model:set batch_size to 1
INFO:tract_linalg.x86_64_fma:qmmm_i32: x86_64/avx2 activated
INFO:tract_linalg.x86_64_fma:mmm_f32, mmv_f32, sigmoid_f32, tanh_f32: x86_64/fma activated
INFO:tract_linalg.x86_64_fma:mmm_f32, mmv_f32: x86_64/avx512f activated
INFO:ezkl.graph.model:set batch_size to 1
INFO:ezkl.graph.model:model has 1 instances
INFO:ezkl.graph.model:calculating num of constraints using dummy model layout...
INFO:ezkl.graph.model:model uses 4065 rows (coord=8131, constants=6259)
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]


gen_settings OK


INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph.model:model has 1 instances
INFO:ezkl.graph.model:calculating num of constraints using dummy model layout...
INFO:ezkl.graph.model:model uses 4065 rows (coord=8131, constants=6259)
INFO:ezkl.graph:setting lookup_range to: (-36, 58), setting logrows to: 13
INFO:ezkl.graph.model:set batch_size to 1
INFO:ezkl.graph.mode

In [9]:
# Compile the model into a circuit
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

INFO:ezkl.graph.model:set batch_size to 1


In [10]:
# Get public srs from kzg ceremony.
res = ezkl.get_srs(settings_path)
assert res == True

INFO:ezkl.execute:SRS does not exist, downloading...
INFO:ezkl.execute:SRS downloaded
INFO:ezkl.execute:SRS hash: d5d94bb25bdc024f649213593027d861042ee807cafd94b49b54f1663f8f267d


In [ ]:
# Setup the circuit and make sure the keys are generated afterwards.
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

vk_size = os.path.getsize(vk_path)
pk_size = os.path.getsize(pk_path)

with open(test_time_and_size_path, 'a') as f:
   f.write(f"Verification key size: {vk_size} bytes\n")
   f.write(f"Proving key size: {pk_size} bytes\n")

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

INFO:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg13.srs"
INFO:ezkl.execute:downsizing params to 13 logrows
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO:ezkl.graph.model:model layout...
INFO:ezkl.graph.model:model uses 3746 rows (coord=7492, constants=5758)
INFO:ezkl.pfsys:VK took 3.21
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO:ezkl.graph.model:model layout...
INFO:ezkl.graph.model:model uses 3746 rows (coord=7492, constants=5758)
INFO:ezkl.pfsys:PK took 1.46
INFO:ezkl.pfsys:saving verification key 💾
INFO:ezkl.pfsys:saving proving key 💾


In [ ]:
# Install solidity compiler
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "solc-select"])
    !solc-select install 0.8.20
    !solc-select use 0.8.20
    !solc --version
# Rely on local installation if the notebook is not in colab
except:
    pass


Installing solc '0.8.20'...
Version '0.8.20' installed.
Switched global version to 0.8.20
solc, the solidity compiler commandline interface
Version: 0.8.20+commit.a1b79de6.Linux.g++


In [ ]:
# Generate the verifier associated with the circuit
res = ezkl.create_evm_verifier(
        vk_path,
        settings_path,
        sol_code_path,
        abi_path
    )
assert res == True
assert os.path.isfile(sol_code_path)

INFO:ezkl.execute:checking solc installation..
INFO:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg13.srs"
INFO:ezkl.execute:downsizing params to 13 logrows
INFO:ezkl.pfsys:loading verification key from "ezkl-outputs/vk.key"
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}


## PROVE AND VERIFY
### Here we will generate and verify a proof locally. Then we will format the inputs and the proof in a way compatible for the evm verifier

In [ ]:
witness_path = os.path.join('ezkl-outputs/witness.json')

# Generate the witness file
res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

INFO:ezkl.graph:input scales: [1, 1]


In [ ]:
# Generate the proof
proof = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

print(proof)
assert os.path.isfile(proof_path)

INFO:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg13.srs"
INFO:ezkl.execute:downsizing params to 13 logrows
INFO:ezkl.pfsys:loading proving key from "ezkl-outputs/pk.key"
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO:ezkl.pfsys:proof started...
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO:ezkl.graph.model:model layout...
INFO:ezkl.graph.model:model uses 3746 rows (coord=7492, constants=5758)
INFO:ezkl.pfsys:proof took 4.843


{'instances': [[[5338117513207086200, 627055957367226120, 2632868032594048778, 2146803414447844525]]], 'proof': '0x0f5129d193f0207a025dd48161ce42eca31b9efc3af36660a11ae737da0616d70379050c780a6266b6678dd2fef54be699c5861e8f1ce1927b20e7c2231506ad0e94b0ba4afa8c1de89af1dc23b2f05b3aa2e8248cd53c7c197a7ba2adb94cd7265390865b74aa25a18dea24961dda2e7efa90c6d4af98ba8ad8d43785ed5e15298606154c8a678902ea927837045e43ebe05146cba581fd4f787df75db7e993115daa83b8920fa73f9714ccaf6fcd4901c4b3ad9c69d1290c00f0cb87ffcfd11e501dca7fcaf9a5dbb7d96146442875be344912beef053e8a4adce3fc6bce6510116561ed739b19cc39bdc3e7842b69a4c6eb017be926f8915bd58688dd48850dda7b585ace11b86d99af89f6e911ab7472d670df1def30603cb8f9346b40ba25cc1674eb3ed716df51079564e85a087c3bf8aa1e9732cb5c399359eb6cd15d2289eab00ce74e19692628a4265d6cc6e417b971bd08da2181f11715db622606012c6e542f4aa029ceb80adb94f4ab420287fd4aa1c00cd9b2a32aa9958d6b140b314217a973fc5cb0ca29e2c871ef5163a20d659c92b803e5a567b491a841851479ed884e96408a7f283438c3fbf7b1ab698fd7b51656bdd1a15

In [ ]:
# Sanity check off-chain verification
res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

assert res == True
print("verified locally")

INFO:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg13.srs"
INFO:ezkl.execute:downsizing params to 13 logrows
INFO:ezkl.pfsys:loading verification key from "ezkl-outputs/vk.key"
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO:ezkl.execute:verify took 0.20
INFO:ezkl.execute:verified: true


verified locally


In [ ]:
# Format the inputs and the proof in a way compatible for the evm verifier
# Get input data for the verifier contract (evmInputs.json)
onchain_input_array = []
# Avoiding printing last comma
formatted_output = "["
for i, value in enumerate(proof["instances"]):
    for j, field_element in enumerate(value):
        onchain_input_array.append(ezkl.vecu64_to_felt(field_element))
        formatted_output += str(onchain_input_array[-1])
        if j != len(value) - 1:
            formatted_output += ", "
    formatted_output += "]"
print("form output",formatted_output )

input_list = [int(entry, 0) for entry in formatted_output.strip('[]').split(', ')]
quoted_list = [format(entry, "#066x") for entry in input_list]
json_data = {
    "instances": quoted_list,
    "proof": proof["proof"]
}
with open('evmInputs.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=2)
print("Saved evmInputs.json: ", json_data)


form output [0x00000000000000000000000000000000000000000000000000000000000000ab]
Saved evmInputs.json:  {'instances': ['0x00000000000000000000000000000000000000000000000000000000000000ab'], 'proof': '0x0f5129d193f0207a025dd48161ce42eca31b9efc3af36660a11ae737da0616d70379050c780a6266b6678dd2fef54be699c5861e8f1ce1927b20e7c2231506ad0e94b0ba4afa8c1de89af1dc23b2f05b3aa2e8248cd53c7c197a7ba2adb94cd7265390865b74aa25a18dea24961dda2e7efa90c6d4af98ba8ad8d43785ed5e15298606154c8a678902ea927837045e43ebe05146cba581fd4f787df75db7e993115daa83b8920fa73f9714ccaf6fcd4901c4b3ad9c69d1290c00f0cb87ffcfd11e501dca7fcaf9a5dbb7d96146442875be344912beef053e8a4adce3fc6bce6510116561ed739b19cc39bdc3e7842b69a4c6eb017be926f8915bd58688dd48850dda7b585ace11b86d99af89f6e911ab7472d670df1def30603cb8f9346b40ba25cc1674eb3ed716df51079564e85a087c3bf8aa1e9732cb5c399359eb6cd15d2289eab00ce74e19692628a4265d6cc6e417b971bd08da2181f11715db622606012c6e542f4aa029ceb80adb94f4ab420287fd4aa1c00cd9b2a32aa9958d6b140b314217a973fc5cb0ca29e2c871ef